# ☀️ Solar Radiation Data Analysis — Benin 🇧🇯


# Table of Contents
- [Introduction](#📊-1-introduction--objective)
<!-- - [Data Loading](## 📦 2. Data Loading & Overview)
- [Exploratory Analysis](## 🔍 3. Missing Values Analysis) -->
- [Conclusion](## 📐 4. Data Types & Basic Stats)
- [Conclusion](## 🧹 5. Data Cleaning)
- [Conclusion](## 📈 6. Univariate Analysis (Single Variable))
- [Conclusion](## 📉 7. Bivariate/Multivariate Analysis)
- [Conclusion](## 📅 9.  Time Series Trends )
- [Conclusion](## 📅 9.  Time Series Trends )
- [Conclusion](## 📅 9.  Time Series Trends )
- [Conclusion](##📅-9.-Time--Series--Trends )


## 📊 1. Introduction & Objective

## 📦 2. Data Loading & Overview

## 🔍 3. Missing Values Analysis

### Summary Statistics & Missing-Value Report

## 📐 4. Data Types & Basic Stats

## 🧹 5. Data Cleaning

### Outlier Detection & Basic Cleaning

### Cleaning Impact

## 📈 6. Univariate Analysis (Single Variable)

## 📉 7. Bivariate/Multivariate Analysis

### Temperature Analysis

### Correlation & Relationship Analysis

### Bubble Chart

### Wind & Distribution Analysis

## 📅 9.  Time Series Trends 

### Time Series Analysis

## 🧠 10. Key Insights

## 🔚 11. Conclusion & Next Steps